# USV - Unmanned Surface Vehicle 
## Magnetometer - Kalibrierung
### Prof. J. Grabow

Das IMU-Magnetometer des USV misst das Erdmagnetfeld in der Einheit Gauss. In Deutschland hat das Erdmagnetfeld ca. eine Feldstärke von 0,48 Gauss. Durch eine Anordnung von drei orthogonal angeordneten Sensoren in der IMU (AK8963) wird ein digitaler Kompass aufgebaut, welcher den Gierwinkel im erdfesten Koordinatensystem erzeugt.

## Fehler des Magnetometers
Wie jeder Sensor weisen auch Magnetometer Fehler auf. Die dabei auftretenden magnetischen Abweichungen (Störfelder), welche den wahren Messwert überlagern, sind sehr groß. Die wichtigsten dieser Störfelder sind Überlagerungen des geomagnetischen Feldes durch Hard- und Soft-Iron Effekte.

### Hard-Iron-Effekte 
Hard-Iron-Effekte entsteht durch statische Magnetfelder wie das Erdmagnetfeld, oder Effekte die vom
USV selbst ausgehen. Das können ferromagnetische Metalle oder Magnete von Antriebsmotoren sein. Solange die Sensorlage nicht verändert wird, bleiben diese Störungen des USV konstant. Auf den Sensorachsen zeigt sich dieser Effekt als konstanter positiver oder negativer Offset.

### Soft-Iron-Effekte
Soft-Iron-Effekte entsteht durch nichtmagnetische Metalle, die jedoch das Erdmagnetfeld stören können. Sie verzerren das örtliche Magnetfeld und führen so zu Fehlern.

In [13]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov  8 13:22:12 2023

Program history
08.11.2023    V. 1.0    Start

@author: Prof. Jörg Grabow (grabow@amesys.de)
"""
__version__ = '1.0'
__author__ = 'Joe Grabow'

import matplotlib.pyplot as plt
import pandas as pd

# load CSV-Datei 
data = pd.read_csv('data.csv', header=None)

# Extrahieren der Spalten x, y und z
x = data[0]
y = data[1]
z = data[2]

# Diagramm xy
plt.figure(figsize=(8, 6))
plt.scatter(x, y, color='blue', label='xy')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title('Diagramm xy')
plt.show()

# Diagramm xz
plt.figure(figsize=(8, 6))
plt.scatter(x, z, color='red', label='xz')
plt.xlabel('x')
plt.ylabel('z')
plt.legend()
plt.title('Diagramm xz')
plt.show()

# Diagramm yz
plt.figure(figsize=(8, 6))
plt.scatter(y, z, color='green', label='yz')
plt.xlabel('y')
plt.ylabel('z')
plt.legend()
plt.title('Diagramm yz')
plt.show()

### Korrektur der Hard-Iron-Effekte 

Die Fehler durch die Hard-Irion-Effekte sind sehr groß aber jedoch auch einfachsten zu korrigieren. Das Verfahren besteht darin, eine Reihe von Magnetometerdaten aufzuzeichnen, während der Sensor langsam in jeder Ebene um 360 Grad gedreht wird. Dabei wird jeweils das Feld in den sechs Hauptrichtungen gemessen wird: +/- Mx, +/- My, +/- Mz. Anschließend werden die jeweiligen Minimal-/Maximalwerte entlang der drei Achsen bestimmt. Wird der Mittelwert von den nachfolgenden Daten subtrahiert, sind die zukünftigen Messdaten wieder auf den Ursprung zu zentriert.

In [14]:
# Hard iron distortion
offset_x = (max(x) + min(x)) / 2
offset_y = (max(y) + min(y)) / 2
offset_z = (max(z) + min(z)) / 2

corrected_x = x - offset_x
corrected_y = y - offset_y
corrected_z = z - offset_z

### Offsetdaten der Hard-Iron-Korrektur

In [15]:

print("offset x: ", offset_x)
print("offset y: ", offset_y)
print("offset z: ", offset_z)

offset x:  18.933985
offset y:  49.1783215
offset z:  -34.304294999999996


### Korrektur der Soft-Iron-Effekte

Um diese Korrektur durchzuführen, müssen von der jeweiligen Messfläche (Ellipse) die Hauptachsen bestimmt werden. Üblicherweise ist dabei eine 3x3 Korrekturmatrix notwendig. Bei dem hier angewendeten Verfahren wird nur eine orthogonale Neuskalierung einer diagonalisierten 3x3 Kalibrierungsmatrix verwendet. Der dabei auftretende Fehler gegenüber der vollbesetzen 3x3 Matrix ist jedoch gering und erlaubt eine geringere Rechenzeit. 


In [16]:
# Soft iron distortion
avg_delta_x = (max(x) - min(x)) / 2
avg_delta_y = (max(y) - min(y)) / 2
avg_delta_z = (max(z) - min(z)) / 2

avg_delta = (avg_delta_x + avg_delta_y + avg_delta_z) / 3

scale_x = avg_delta / avg_delta_x
scale_y = avg_delta / avg_delta_y
scale_z = avg_delta / avg_delta_z

### Skalierungsfaktor der Soft-Iron-Korrektur

In [17]:
print("scale x: ", scale_x)
print("scale y: ", scale_y)
print("scale z: ", scale_z)

scale x:  1.0107710192152866
scale y:  1.047377406882184
scale z:  0.9470678452641178


### Korrektur der Messdaten

In [18]:
corrected_x = (x - offset_x) * scale_x
corrected_y = (y - offset_y) * scale_y
corrected_z = (z - offset_z) * scale_z

# Diagramm scale xy
plt.figure(figsize=(8, 8))
plt.scatter(corrected_x, corrected_y, color='blue', label='xy')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title('Diagramm scale xy')
plt.show()

# Diagramm scale xz
plt.figure(figsize=(8, 8))
plt.scatter(corrected_x, corrected_z, color='red', label='xz')
plt.xlabel('x')
plt.ylabel('z')
plt.legend()
plt.title('Diagramm scale xz')
plt.show()

# Diagramm scale yz
plt.figure(figsize=(8, 8))
plt.scatter(corrected_y, corrected_z, color='green', label='yz')
plt.xlabel('y')
plt.ylabel('z')
plt.legend()
plt.title('Diagramm scale yz')
plt.show()